In [ ]:
#pdf_folder_path = '/content/drive/MyDrive/Colab Notebooks/dataset'

In [56]:
!pip install elasticsearch
!pip install transformers torch
!pip install fastapi uvicorn elasticsearch

###elasticSearch에서 벡터필드를 포함한 인덱스 생성

In [ ]:
# 매핑 수정
index_mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "body": {"type": "text"},
            "tag": {"type": "keyword"},
            "deadline": {"type": "date"},
            "period": {"type": "text"},
            "type": {"type": "keyword"},
            "embedding": {"type": "dense_vector", "dims": 384}  # 임베딩 차원 수정
        }
    }
}

In [ ]:
# 새로운 인덱스 생성
index_name = "job_and_post"
es.indices.create(index=index_name, body=index_mapping, ignore=400)

<ipython-input-11-32aee0870fb2>:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, body=index_mapping, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'job_and_post'})

###elastic search 검색 test (벡터 검색쿼리 )

##FASTAPI 로 배포

In [84]:
from elasticsearch import Elasticsearch

# Elastic Cloud 정보
CLOUD_ID = "capstoneAI:"
USERNAME = "elastic"
PASSWORD = ""

# 클라이언트 생성
es = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=(USERNAME, PASSWORD)
)

# 연결 확인
if es.ping():
    print("ElasticSearch 연결 성공")
else:
    print("ElasticSearch 연결 실패")


ElasticSearch 연결 성공


In [58]:
!pip install python-multipart

In [59]:
!pip install pyngrok fastapi

In [60]:
!pip install nest_asyncio uvicorn

In [61]:
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import nest_asyncio
import uvicorn

In [62]:
ngrok.set_auth_token("")

In [63]:
#벡터 임베딩 생성
from transformers import AutoTokenizer, AutoModel
import torch

# BERT 모델 로드
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-51' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", 

In [64]:
# 텍스트 임베딩 생성 함수
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().tolist()

In [65]:
from fastapi import FastAPI
from elasticsearch import Elasticsearch

app = FastAPI()

# Elasticsearch 연결
es = Elasticsearch(
    cloud_id="capstoneAI:",
    basic_auth=("elastic", ""))
index_name = "job_and_post"

In [66]:
# CORS 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    # "http://127.0.0.1:5500",  # 허용할 출처 설정
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
# # 데이터 색인
# @app.post("/create/")
# async def create_document(doc: dict):
#     # Ensure 필드
#     if "title" in doc and "body" in doc:
#         doc["embedding"] = get_embedding(doc["title"] + " " + doc["body"])
#     else:
#         return {"error": "Title and body fields are required for embedding."}

#     # Save document to ElasticSearch
#     res = es.index(index=index_name, body=doc)
#     return {"message": "Document indexed", "id": res["_id"]}


In [83]:
from random import sample
from fastapi import FastAPI, Query
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from elasticsearch import Elasticsearch
from fastapi import FastAPI, Query, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import List
import json

class SearchRequest(BaseModel):
    title: str
    tags: List[str] = []

@app.post("/search")
async def search_documents(data: SearchRequest):
    query = data.title
    tags = data.tags
    size = 3  # 결과 수 조정 가능

    if query or tags:
        query_vector = get_embedding(query + " " + " ".join(tags))
        search_query = {
            "query": {
                "script_score": {
                    "query": {"match_all": {}},
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                        "params": {"query_vector": query_vector},
                    },
                },
            },
            "size": size * 2,
        }

        response = es.search(index=index_name, body=search_query)
        all_results = response["hits"]["hits"]
        job_results = [doc for doc in all_results if doc["_source"].get("type") == "job"]
        portfolio_results = [doc for doc in all_results if doc["_source"].get("type") == "portfolio"]

        sorted_job_results = sorted(job_results, key=lambda x: x["_score"], reverse=True)[:size]
        sorted_portfolio_results = sorted(portfolio_results, key=lambda x: x["_score"], reverse=True)[:size]

        return JSONResponse(
    content={
        "jobs": [
            {
                "id": doc["_id"],
                "title": doc["_source"].get("title"),
                "body": doc["_source"].get("body"),
                "author": doc["_source"].get("author"),
                "tag": doc["_source"].get("tag"),
                "period": doc["_source"].get("period"),
            }
            for doc in sorted_job_results
        ] if sorted_job_results else [],
        "portfolio": [
            {
                "id": doc["_id"],
                "title": doc["_source"].get("title"),
                "body": doc["_source"].get("body"),
                "author": doc["_source"].get("author"),
                "tag": doc["_source"].get("tag"),
                "period": doc["_source"].get("period"),
            }
            for doc in sorted_portfolio_results
        ] if sorted_portfolio_results else [],
    }
)
    return JSONResponse(content={"jobs": [], "portfolio": []})

In [85]:
nest_asyncio.apply()

In [86]:
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# FastAPI 서버 실행
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://bdf9-34-91-153-171.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [157]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2001:2d8:e723:ad7b:1d62:17a9:bf00:dbd7:0 - "OPTIONS /search HTTP/1.1" 200 OK
INFO:     2001:2d8:e723:ad7b:1d62:17a9:bf00:dbd7:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2001:2d8:e723:ad7b:1d62:17a9:bf00:dbd7:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2001:2d8:e723:ad7b:1d62:17a9:bf00:dbd7:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2001:2d8:e723:ad7b:1d62:17a9:bf00:dbd7:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2001:2d8:e723:ad7b:1d62:17a9:bf00:dbd7:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2001:2d8:e723:ad7b:1d62:17a9:bf00:dbd7:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2001:2d8:e723:ad7b:1d62:17a9:bf00:dbd7:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2001:2d8:e723:ad7b:1d62:17a9:bf00:dbd7:0 - "POST /search HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [157]


###NLP 모델 연결

In [80]:
job_docs = [
    {
        "title": "머신러닝 엔지니어 채용",
        "body": (
            "AI 스타트업 'AI Technologies'에서 머신러닝 엔지니어를 모집합니다. "
            "서울 본사에서 근무하며, 주요 업무는 머신러닝 모델 개발 및 데이터 분석입니다. "
            "필수 조건: Python, TensorFlow, 데이터 전처리 경험. "
            "우대 사항: 클라우드 플랫폼(GCP, AWS) 경험. "
            "근무 형태: 정규직. 급여는 경력에 따라 협의."
        ),
        "tag": ["꼼꼼한 성격", "책임감"],
        "deadline": "2024-12-31",
        "author": "AI Technologies",
        "type": "job"
    },
    {
        "title": "React 프론트엔드 개발자 모집",
        "body": (
            "전자상거래 플랫폼 'E-Shop'에서 React 프론트엔드 개발자를 모집합니다. "
            "서울 또는 원격 근무 가능하며, 주요 업무는 React 기반 웹 애플리케이션 개발입니다. "
            "필수 조건: React, Redux, RESTful API 경험. "
            "우대 사항: TypeScript 및 AWS 경험. "
            "근무 형태: 계약직(6개월), 이후 연장 가능."
        ),
        "tag": ["책임감", "빠른 일처리"],
        "deadline": "2024-11-30",
        "author": "E-Shop",
        "type": "job"
    },
    {
        "title": "데이터 분석가 모집",
        "body": (
            "IT 컨설팅 기업 'DataWise'에서 데이터 분석가를 모집합니다. "
            "주요 업무는 Python과 SQL을 활용한 데이터 분석, 대시보드 개발, 비즈니스 인사이트 도출입니다. "
            "필수 조건: 데이터 분석 경험, SQL 및 Excel 고급 스킬. "
            "우대 사항: Tableau 및 Power BI 경험. "
            "근무 형태: 정규직. 근무지는 부산이며, 재택 근무 협의 가능."
        ),
        "tag": ["팀워크", "성실"],
        "deadline": "2024-12-15",
        "author": "DataWise",
        "type": "job"
    },
    {
        "title": "디지털 마케팅 인턴 모집",
        "body": (
            "글로벌 광고 대행사 'AdMax'에서 디지털 마케팅 인턴을 모집합니다. "
            "주요 업무는 소셜 미디어 콘텐츠 기획, 광고 캠페인 데이터 분석, 마케팅 전략 수립 지원입니다. "
            "필수 조건: 소셜 미디어 운영 경험, Excel 능숙. "
            "우대 사항: Google Ads 인증."
        ),
        "tag": ["열정", "창의성"],
        "deadline": "2024-10-31",
        "author": "AdMax",
        "type": "job"
    },
    {
        "title": "UX/UI 디자이너 채용",
        "body": (
            "'CreativeLab'에서 UX/UI 디자이너를 모집합니다. "
            "주요 업무는 사용자 경험 연구, UI 디자인 및 프로토타입 제작입니다. "
            "필수 조건: Figma 또는 Sketch 숙련자, 사용자 경험 개선 프로젝트 경험. "
            "우대 사항: 스타트업 환경에서의 근무 경험."
        ),
        "tag": ["섬세함", "창의성"],
        "deadline": "2024-11-15",
        "author": "CreativeLab",
        "type": "job"
    },
    {
        "title": "AI 챌린지 공모전 참가자 모집",
        "body": (
            "'AI Challenge 2024' 공모전에 참가할 팀원을 모집합니다. "
            "머신러닝과 딥러닝 프로젝트 경험이 있는 개발자를 우대하며, 주제는 자연어 처리와 추천 시스템입니다."
        ),
        "tag": ["팀워크", "책임감"],
        "deadline": "2024-09-30",
        "author": "AI Challenge 2024",
        "type": "job"
    },
    {
        "title": "로봇 공학 인턴십",
        "body": (
            "로봇 솔루션 기업 'RoboTech'에서 로봇 공학 인턴을 모집합니다. "
            "근무지는 대전이며, 주요 업무는 로봇 제어 알고리즘 설계 및 테스트입니다. "
            "필수 조건: C++ 및 ROS 경험. "
            "우대 사항: 로봇 시뮬레이션 경험."
        ),
        "tag": ["창의성", "꼼꼼한 성격"],
        "deadline": "2024-08-31",
        "author": "RoboTech",
        "type": "job"
    },
    {
        "title": "웹 개발자 채용",
        "body": (
            "중견 IT 기업 'WebCore'에서 웹 개발자를 모집합니다. "
            "주요 업무는 Django 기반 웹 애플리케이션 개발 및 유지보수입니다. "
            "필수 조건: Django, PostgreSQL 경험. "
            "우대 사항: 클라우드 환경에서의 개발 경험."
        ),
        "tag": ["책임감", "빠른 일처리"],
        "deadline": "2024-10-15",
        "author": "WebCore",
        "type": "job"
    },
    {
        "title": "국제 비즈니스 대회 참가자 모집",
        "body": (
            "국제 비즈니스 아이디어 대회에 참가할 팀원을 모집합니다. "
            "주요 역할은 비즈니스 모델 기획 및 발표 자료 제작입니다. "
            "MBA 학생 또는 비즈니스 컨설팅 경험자를 우대합니다."
        ),
        "tag": ["리더십", "팀워크"],
        "deadline": "2024-09-15",
        "author": "Global Biz Challenge",
        "type": "job"
    },
    {
        "title": "에너지 공학 인턴십",
        "body": (
            "'Green Energy Solutions'에서 에너지 공학 인턴을 모집합니다. "
            "주요 업무는 재생 에너지 프로젝트 지원, 데이터 분석 및 보고서 작성입니다. "
            "필수 조건: MATLAB 또는 Python 사용 경험. "
            "우대 사항: 에너지 저장 시스템 설계 경험."
        ),
        "tag": ["성실", "분석력"],
        "deadline": "2024-12-01",
        "author": "Green Energy Solutions",
        "type": "job"
    }
]

# title과 body를 결합해 embedding 생성
for doc in job_docs:
    doc["embedding"] = get_embedding(doc["title"] + " " + doc["body"])


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-75' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", 

In [81]:
post_docs = [
    {
        "title": "제10회 인문학 공모전 참여",
        "body": (
            "제10회 인문학 공모전에 참여하여 '문학 속 사랑의 표현'이라는 주제로 작품을 제출했습니다. "
            "해당 공모전에서는 창의성과 논리적인 글쓰기 능력을 인정받아 우수상을 수상했습니다."
        ),
        "period": "2023-04 ~ 2023-05",
        "tag": ["창의성", "논리적 사고"],
        "author": "김문학",
        "type": "portfolio"
    },
    {
        "title": "IT 공모전에서 AI 프로젝트 발표",
        "body": (
            "'Smart AI' 공모전에 참가하여 머신러닝 기반 추천 시스템을 발표했습니다. "
            "데이터 수집부터 모델 개발, 시각화까지 팀을 이끌어 프로젝트를 성공적으로 완수했으며, 최우수상을 수상했습니다."
        ),
        "period": "2022-09 ~ 2022-12",
        "tag": ["팀워크", "책임감"],
        "author": "박인공지능",
        "type": "portfolio"
    },
    {
        "title": "환경 캠페인 참여",
        "body": (
            "환경 보호 단체와 협력하여 '플라스틱 줄이기 캠페인'에 참여했습니다. "
            "캠페인의 기획부터 실행까지 전 과정에 참여했으며, 지역 주민의 참여를 독려하는 활동을 주도했습니다."
        ),
        "period": "2023-01 ~ 2023-03",
        "tag": ["열정", "소통"],
        "author": "이환경",
        "type": "portfolio"
    },
    {
        "title": "스타트업 창업 경험",
        "body": (
            "2021년에 IT 기반 스타트업을 창업하여, 팀 빌딩부터 제품 개발, 투자 유치까지 전 과정을 경험했습니다. "
            "현재는 서비스를 성공적으로 운영하고 있으며, 사용자 기반이 꾸준히 성장하고 있습니다."
        ),
        "period": "2021-01 ~ 현재",
        "tag": ["리더십", "창의성"],
        "author": "최창업",
        "type": "portfolio"
    },
    {
        "title": "LLM RAG 개발 및 삼성 프로젝트 활동",
        "body": (
            "삼성전자와의 협력 프로젝트에서 LLM 기반의 RAG(Retrieval-Augmented Generation) 시스템을 개발했습니다. "
            "프로젝트는 데이터 검색 및 자연어 처리 시스템 최적화를 목표로 진행되었으며, 성공적으로 완료되었습니다."
        ),
        "period": "2023-05 ~ 2023-10",
        "tag": ["논리적 사고", "책임감"],
        "author": "홍삼성",
        "type": "portfolio"
    },
    {
        "title": "UI/UX 디자인 프로젝트",
        "body": (
            "대학 졸업 프로젝트로 UI/UX 디자인에 참여하여 사용자 친화적인 모바일 앱을 개발했습니다. "
            "해당 앱은 디자인 공모전에서 1위를 차지했으며, 사용자 경험 테스트를 통해 지속적으로 개선되었습니다."
        ),
        "period": "2022-03 ~ 2022-11",
        "tag": ["창의성", "섬세함"],
        "author": "유아이",
        "type": "portfolio"
    },
    {
        "title": "OpenAI 기반 챗봇 개발",
        "body": (
            "OpenAI의 GPT-3 API를 활용하여 고객 지원용 챗봇을 개발한 경험이 있습니다. "
            "해당 챗봇은 자연어 이해 성능을 극대화하여 고객의 요청을 신속히 처리할 수 있도록 설계되었습니다."
        ),
        "period": "2023-06 ~ 2023-08",
        "tag": ["빠른 일처리", "창의성"],
        "author": "박챗봇",
        "type": "portfolio"
    },
    {
        "title": "하나은행 대규모 금융 데이터 분석 프로젝트",
        "body": (
            "금융권 고객사의 요구에 맞춰 대규모 데이터를 분석하고 시각화하는 프로젝트를 수행했습니다. "
            "Python과 Tableau를 활용하여 실시간 대시보드를 제작하고, 핵심 인사이트를 도출했습니다."
        ),
        "period": "2023-01 ~ 2023-04",
        "tag": ["분석력", "책임감"],
        "author": "김데이터",
        "type": "portfolio"
    },
    {
        "title": "사회적 기업 캠페인 기획 및 실행",
        "body": (
            "사회적 기업과 협력하여 청소년 대상 코딩 교육 캠페인을 기획하고 실행했습니다. "
            "100명 이상의 참가자를 대상으로 성공적인 교육 프로그램을 운영했습니다."
        ),
        "period": "2022-07 ~ 2022-09",
        "tag": ["소통", "열정"],
        "author": "강교육",
        "type": "portfolio"
    },
    {
        "title": "유니티활용 모바일 게임 개발 및 출시",
        "body": (
            "Unity 엔진을 활용하여 모바일 게임을 개발하고 Google Play 스토어에 출시했습니다. "
            "게임 디자인부터 프로그래밍, 테스트까지 전 과정을 담당하며 10,000+ 다운로드를 달성했습니다."
        ),
        "period": "2022-01 ~ 2022-06",
        "tag": ["리더십", "창의성"],
        "author": "최게임",
        "type": "portfolio"
    }
]


# title과 body를 결합해 embedding 생성
for doc in post_docs:
    doc["embedding"] = get_embedding(doc["title"] + " " + doc["body"])


In [82]:
# ElasticSearch 색인
for i, doc in enumerate(job_docs):
    es.index(index=index_name, id=f"job_{i+1}", body=doc)

for i, doc in enumerate(post_docs):
    es.index(index=index_name, id=f"post_{i+1}", body=doc)


In [54]:
# ElasticSearch 색인
for i, doc in enumerate(post_docs):
    es.index(index=index_name, id=f"job_{i+1}", body=doc)

print(f"{len(post_docs)}개의 포트폴리오가 색인되었습니다.")

10개의 포트폴리오가 색인되었습니다.


In [75]:
def search_combined_indices(query, tags, top_k=3):
    # 1. 사용자가 입력한 검색어+태그 임베딩 생성
    query_embedding = get_embedding(query + " " + " ".join(tags)).tolist()

    # 2. ElasticSearch에서 두 인덱스를 각각 검색
    indices = ["jobs_docs", "post_docs"]
    all_results = []

    for index in indices:
        query_body = {
            "query": {
                "script_score": {
                    "query": {"match_all": {}},
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                        "params": {"query_vector": query_embedding}
                    }
                }
            },
            "size": top_k
        }
        response = es.search(index=index, body=query_body)
        # 각 결과를 리스트에 추가
        all_results.extend(response["hits"]["hits"])

    # 3. 유사도 기준으로 정렬
    sorted_results = sorted(all_results, key=lambda x: x["_score"], reverse=True)
    return sorted_results
